In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

from ibm_botocore.client import Config
import ibm_boto3
import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F


import numpy as np
import pandas as pd
import json
import pyspark.sql
from pyspark.sql.functions import col, skewness, kurtosis
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import when
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from datetime import date, timedelta, datetime
import time

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210210213727-0000
KERNEL_ID = a7dcacf9-e277-42da-99ae-cd4c2ca6bb0f


Hidden cell with credentials for accessing from Cloud Object Storage

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
#cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')
#ratings_df = spark.read.parquet(cos.url('ratingsdataframe.parquet', <BUCKET NAME>))

ratings_df.createOrReplaceTempView("ratings")

In [4]:
result = spark.sql("""
            Select rating as Ratings,COUNT(rating) as Number_of_Movies_per_rating
            FROM ratings 
            GROUP BY rating
            ORDER BY rating ASC
            """).repartition(1)

result.show()


+-------+---------------------------+
|Ratings|Number_of_Movies_per_rating|
+-------+---------------------------+
|      1|                      56174|
|      2|                     107557|
|      3|                     261197|
|      4|                     348971|
|      5|                     226310|
+-------+---------------------------+



In [5]:
## To save it locally
from IPython.display import HTML 
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "result.csv"):  
    csv = df.to_csv(sep=',',header=True, index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# sparkDF to Pandas
result_df = result.toPandas()
create_download_link(result_df,filename='movies_per_rating.csv')